<a href="https://colab.research.google.com/github/IsaacSebastian/ingCaracteristicas/blob/KevinG/%7B%7B%20cookiecutter.project_slug%20%7D%7D/notebooks/Ensanut_datos_completos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Datos finales de salud mental

En esta libreta uniremos los datos de las encuestas de la ENSANUT de adolescentes y adultos.

Comenzamos por importar las librerias necesarias.

In [40]:
import pandas as pd
import os
import geopandas as gpd
import palettable as pltt
from seaborn import palplot
import requests
import zipfile

Jalamos los archivos desde la carpeta interim

In [41]:
file_path_adol = '/content/Adolescentes.csv'
file_path_adul = '/content/Adultos.csv'

Concatenamos los dataframes.

In [42]:
df_adol = pd.read_csv(file_path_adol, low_memory=False)
df_adul = pd.read_csv(file_path_adul, low_memory=False)
dataframes = [df_adol,df_adul]  # Reemplaza con los nombres de tus DataFrames
# Concatenar los DataFrames hacia abajo
df_concatenado = pd.concat(dataframes, axis=0, ignore_index=True)

Guardamos en el directorio interim

In [43]:
# ruta donde guardar el CSV
file_path = '/content/Datos-Adol-Adul.csv'

# Guarda el DataFrame en formato CSV
df_concatenado.to_csv(file_path, index=False)  # index=False evita que se guarde el índice del DataFrame

Dado que ya tenemos los datos es interim y descargados los datos del mapa, podemos hacer un merge para ponerle la columna del nombre de estado a nuestros datos y terminar el procesamiento.

In [44]:
datos_mapa = gpd.read_file(mapa_a)
file_path = '/content/Datos-Adol-Adul.csv'
datos_ensa = gpd.read_file(file_path)
datos_mapa.head()

datos_ensa['Entidad'] = datos_ensa['Entidad'].astype(int)
datos_mapa['CVE_ENT'] = datos_mapa['CVE_ENT'].astype(int)

# Fusionar los DataFrames en la columna 'Entidad' (CSV) y 'CVE_ENT' (SHP)
df_combinado = pd.merge(datos_ensa, datos_mapa, left_on='Entidad', right_on='CVE_ENT', how='left')

df_combinado = gpd.GeoDataFrame(df_combinado, geometry='geometry')

df_combinado = df_combinado.drop(columns=['CVE_ENT','CVEGEO', 'geometry'])
df_combinado  = df_combinado.rename(columns={'NOMGEO': 'Entidad', 'Entidad':'C_Entidad'})

df_combinado = df_combinado[['Edad',	'Sexo',	'C_Entidad', 'Entidad',	'Fecha',	'Atentar_contras_si',	'Depresion',	'Tristeza']]

In [45]:
df_combinado

,Edad,Sexo,C_Entidad,Entidad,Fecha,Atentar_contras_si,Depresion,Tristeza
0,11,1,1,Aguascalientes,2006-07-01,3,0,0
1,18,2,1,Aguascalientes,2006-07-01,3,0,0
2,17,2,1,Aguascalientes,2006-07-01,3,0,0
3,10,1,1,Aguascalientes,2006-07-01,3,0,0
4,14,2,1,Aguascalientes,2006-07-01,3,0,0
...,...,...,...,...,...,...,...,...
259698,50,2,32,Zacatecas,2021-09-26,3,4,2
259699,31,1,32,Zacatecas,2021-09-24,3,1,1
259700,27,1,32,Zacatecas,2021-09-24,3,1,1
259701,39,1,32,Zacatecas,2021-09-24,3,1,1


Revisamos si hay datos perdidos.

In [31]:
# Para ver los datos perdidos en df_combinado
missing_data = df_combinado.isnull().sum()

# Crear un DataFrame para mostrar la información de manera más clara
missing_data_df = pd.DataFrame(missing_data, columns=['Missing Values'])
missing_data_df


,Missing Values
Edad,0
Sexo,0
C_Entidad,0
Entidad,0
Fecha,0
Atentar_contras_si,0
Depresion,0
Tristeza,0


Verificamos cuantos datos unicos hay.

In [32]:
# Iterar sobre cada columna y mostrar los valores únicos
for column in df_combinado.columns:
    unique_values = df_combinado[column].unique()
    print(f"Valores únicos en la columna '{column}': {unique_values}\n")


Valores únicos en la columna 'Edad': ['11' '18' '17' '10' '14' '13' '15' '12' '19' '16' '27' '21' '39' '61'
 '28' '34' '23' '43' '57' '33' '35' '46' '31' '24' '22' '38' '36' '80'
 '25' '20' '40' '41' '37' '58' '30' '64' '52' '54' '78' '47' '42' '75'
 '59' '49' '69' '51' '86' '48' '26' '70' '44' '50' '32' '45' '76' '63'
 '74' '65' '68' '72' '85' '53' '82' '84' '87' '79' '73' '60' '83' '29'
 '81' '71' '56' '62' '66' '91' '55' '67' '77' '97' '92' '89' '90' '94'
 '99' '88' '93' '96' '95' '98' '101' '100' '104' '102' '114' '109' '105'
 '111' '110' '112']

Valores únicos en la columna 'Sexo': ['1' '2']

Valores únicos en la columna 'C_Entidad': [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32]

Valores únicos en la columna 'Entidad': ['Aguascalientes' 'Baja California' 'Baja California Sur' 'Campeche'
 'Coahuila de Zaragoza' 'Colima' 'Chiapas' 'Chihuahua' 'Ciudad de México'
 'Durango' 'Guanajuato' 'Guerrero' 'Hidalgo' 'Jalisco' 'México'
 'Mich

Dado que hemos utilizado el 0 como dato perdido entonces los 9 y ' ', los transformamos a 0 en las columnas que lo requieren. Posteriormente verificamos los cambios.

In [33]:
# Reemplazar valores en las columnas especificadas
df_combinado['Atentar_contras_si'] = df_combinado['Atentar_contras_si'].replace(['9', ' '], 0)
df_combinado['Depresion'] = df_combinado['Depresion'].replace(['9', ' '], 0)
df_combinado['Tristeza'] = df_combinado['Tristeza'].replace(['9', ' '], 0)

# Asegurarse de que las columnas sean de tipo numérico
df_combinado['Atentar_contras_si'] = pd.to_numeric(df_combinado['Atentar_contras_si'], errors='coerce')
df_combinado['Depresion'] = pd.to_numeric(df_combinado['Depresion'], errors='coerce')
df_combinado['Tristeza'] = pd.to_numeric(df_combinado['Tristeza'], errors='coerce')

# Verificar los valores únicos nuevamente para asegurarse de que los cambios se realizaron
for column in ['Atentar_contras_si', 'Depresion', 'Tristeza']:
    unique_values = df_combinado[column].unique()
    print(f"Valores únicos en la columna '{column}': {unique_values}\n")


Valores únicos en la columna 'Atentar_contras_si': [3 2 1 0]

Valores únicos en la columna 'Depresion': [0 1 2 3 4]

Valores únicos en la columna 'Tristeza': [0 1 2 3 4]



Ahora nos dams cuenta que los tipos de datos son incorrectos.

In [34]:
# Mostrar los tipos de datos de cada columna en df_combinado
print(df_combinado.dtypes)

Edad                  object
Sexo                  object
C_Entidad              int64
Entidad               object
Fecha                 object
Atentar_contras_si     int64
Depresion              int64
Tristeza               int64
dtype: object


Cambiamos el tipo de dato al adecuado.

In [35]:
# Convertir la columna 'Edad' a tipo entero
df_combinado['Edad'] = df_combinado['Edad'].astype(int)

# Convertir la columna 'Sexo' a tipo categórico
df_combinado['Sexo'] = df_combinado['Sexo'].astype('category')

# Convertir la columna 'Entidad' a tipo categórico
df_combinado['Entidad'] = df_combinado['Entidad'].astype('category')

# Convertir la columna 'Fecha' a tipo datetime
df_combinado['Fecha'] = pd.to_datetime(df_combinado['Fecha'], errors='coerce')

# Convertir las columnas 'Atentar_contras_si', 'Depresion' y 'Tristeza' a tipo categórico
df_combinado['Atentar_contras_si'] = df_combinado['Atentar_contras_si'].astype('category')
df_combinado['Depresion'] = df_combinado['Depresion'].astype('category')
df_combinado['Tristeza'] = df_combinado['Tristeza'].astype('category')

# Verificar los tipos de datos después de las conversiones
print(df_combinado.dtypes)


Edad                           int64
Sexo                        category
C_Entidad                      int64
Entidad                     category
Fecha                 datetime64[ns]
Atentar_contras_si          category
Depresion                   category
Tristeza                    category
dtype: object


Por ultimo, veamos cuantos de los datos osn 0, esto nos da la cantidad de datos perdidos por columna.

In [37]:
# Contar ceros en cada columna del DataFrame
zero_counts = (df_combinado == 0).sum()

# Mostrar solo las columnas que tienen ceros
zero_counts = zero_counts[zero_counts > 0]

print("Conteo de ceros en cada columna:\n", zero_counts)

Conteo de ceros en cada columna:
 Atentar_contras_si     99687
Depresion             103833
Tristeza               58632
dtype: int64


Como se esperaba, debido a la natulraleza de los datos, al ser entrevista presentan muchos datos perdidos en las columnas de las preguntas, pues no todos los años se siguio una encuesta estandar. Fue hasta del año 2021 que se establecio una y esta se siguio en 2022 y 2023.


Por ultimo, una ultima vista a nuestros datos procesados.

In [38]:
df_combinado

,Edad,Sexo,C_Entidad,Entidad,Fecha,Atentar_contras_si,Depresion,Tristeza
0,11,1,1,Aguascalientes,2006-07-01,3,0,0
1,18,2,1,Aguascalientes,2006-07-01,3,0,0
2,17,2,1,Aguascalientes,2006-07-01,3,0,0
3,10,1,1,Aguascalientes,2006-07-01,3,0,0
4,14,2,1,Aguascalientes,2006-07-01,3,0,0
...,...,...,...,...,...,...,...,...
259698,50,2,32,Zacatecas,2021-09-26,3,4,2
259699,31,1,32,Zacatecas,2021-09-24,3,1,1
259700,27,1,32,Zacatecas,2021-09-24,3,1,1
259701,39,1,32,Zacatecas,2021-09-24,3,1,1


Terminamos guardando los datos en processed, faltando solamente pasar por una revision de calidad de los datos.

In [39]:
df_combinado.to_csv('df_combinado.csv', index=False)